In [33]:
from utils import CovidMongo
import pandas as pd
import os
import numpy as np
import json
import re
import os

# --------- Pandas Settings ---------- #
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', -1)

/home/ubuntu/.local/share/virtualenvs/covid19-dash-q8XJLdOW/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  if sys.path[0] == '':


## Load Data

In [34]:
def find_csvs(path):
    csv_files = []
    directory = os.path.join("/home/ubuntu/projects/covid19-dash", path)
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".csv"):
                csv_files.append(os.path.join(directory, file))
    return csv_files


csv_files = find_csvs("csse_covid_19_data/csse_covid_19_daily_reports")
csv_files[1]

'/home/ubuntu/projects/covid19-dash/csse_covid_19_data/csse_covid_19_daily_reports/02-25-2020.csv'

In [35]:
def load_csvs(csv_files):
    dfs = []
    for csv_file in csv_files:
        df = pd.read_csv(csv_file)
        dfs.append(df)
    df_total = pd.concat(dfs, ignore_index=True)
    return df_total

df = load_csvs(csv_files)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5890 entries, 0 to 5889
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Province/State  3575 non-null   object 
 1   Country/Region  5890 non-null   object 
 2   Last Update     5890 non-null   object 
 3   Confirmed       5871 non-null   float64
 4   Deaths          5449 non-null   float64
 5   Recovered       5502 non-null   float64
 6   Latitude        3072 non-null   float64
 7   Longitude       3072 non-null   float64
dtypes: float64(5), object(3)
memory usage: 368.2+ KB


In [36]:
df.head()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude
0,Anhui,Mainland China,1/23/20 17:00,9.0,NaN,NaN,NaN,NaN
1,Beijing,Mainland China,1/23/20 17:00,22.0,NaN,NaN,NaN,NaN
2,Chongqing,Mainland China,1/23/20 17:00,9.0,NaN,NaN,NaN,NaN
3,Fujian,Mainland China,1/23/20 17:00,5.0,NaN,NaN,NaN,NaN
4,Gansu,Mainland China,1/23/20 17:00,2.0,NaN,NaN,NaN,NaN


## Extract US data

In [37]:
df_usa = df[df['Country/Region'] == 'US']
df_usa

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude
31,Washington,US,1/23/20 17:00,1.0,NaN,NaN,NaN,NaN
83,Unassigned Location (From Diamond Princess),US,2020-02-25T21:23:03,36.0,0.0,0.0,NaN,NaN
108,"Chicago, IL",US,2020-02-09T19:03:03,2.0,0.0,2.0,NaN,NaN
109,"San Benito, CA",US,2020-02-03T03:53:02,2.0,0.0,0.0,NaN,NaN
110,"San Diego County, CA",US,2020-02-21T05:43:02,2.0,0.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...
5885,"Los Angeles, CA",US,2020-02-01T19:53:03,1.0,0.0,0.0,NaN,NaN
5886,"Madison, WI",US,2020-02-05T21:53:02,1.0,0.0,0.0,NaN,NaN
5887,"Orange, CA",US,2020-02-01T19:53:03,1.0,0.0,0.0,NaN,NaN
5888,"Seattle, WA",US,2020-02-09T07:03:04,1.0,0.0,1.0,NaN,NaN


In [26]:
df_usa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1218 entries, 31 to 5889
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Province/State  1218 non-null   object 
 1   Country/Region  1218 non-null   object 
 2   Last Update     1218 non-null   object 
 3   Confirmed       1218 non-null   float64
 4   Deaths          1188 non-null   float64
 5   Recovered       1188 non-null   float64
 6   Latitude        844 non-null    float64
 7   Longitude       844 non-null    float64
dtypes: float64(5), object(3)
memory usage: 85.6+ KB


### Extract US States

In [52]:
df_usa['Province/State'].value_counts()

Los Angeles, CA                                38
San Benito, CA                                 36
Tempe, AZ                                      35
Santa Clara, CA                                35
Madison, WI                                    34
Boston, MA                                     34
Orange, CA                                     32
Seattle, WA                                    30
Chicago, IL                                    30
San Diego County, CA                           28
San Antonio, TX                                26
Sacramento County, CA                          18
Humboldt County, CA                            18
Lackland, TX (From Diamond Princess)           17
Omaha, NE (From Diamond Princess)              17
Travis, CA (From Diamond Princess)             17
Washington                                     16
Unassigned Location (From Diamond Princess)    15
Illinois                                       13
California                                     12


### Find NAs

In [41]:
df_usa.isna().sum()

Province/State    0  
Country/Region    0  
Last Update       0  
Confirmed         0  
Deaths            30 
Recovered         30 
Latitude          374
Longitude         374
dtype: int64

## Remove Diamond Princess

In [49]:
df["Province/State"].str.lower().str.startswith("diamond")

2315